[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

LangChain Academyへようこそ！

## 背景

LangChainでは、LLMアプリケーションの構築を簡単にすることを目指しています。構築できるLLMアプリケーションの一つにエージェントがあります。エージェントは以前では不可能だった幅広いタスクを自動化できるため、その構築に大きな注目が集まっています。

しかし実際には、これらのタスクを確実に実行するシステムを構築することは非常に困難です。私たちがユーザーと協力してエージェントを本番環境に導入する中で、より多くの制御が必要であることを学びました。エージェントに常に特定のツールを最初に呼び出させたり、状態に応じて異なるプロンプトを使用させる必要がある場合があります。

この問題に対処するため、私たちは[LangGraph](https://langchain-ai.github.io/langgraph/)を構築しました。これはエージェントおよびマルチエージェントアプリケーションを構築するためのフレームワークです。LangChainパッケージとは別に、LangGraphの核となる設計思想は、開発者がエージェントワークフローにより良い精度と制御を追加し、実世界のシステムの複雑さに適した開発を支援することです。

## コース構成

コースは一連のモジュールとして構成されており、各モジュールはLangGraphに関連する特定のテーマに焦点を当てています。各モジュールのフォルダーには一連のノートブックが含まれています。各ノートブックには概念を説明するビデオが付随していますが、ノートブック自体も独立しており、説明が含まれているためビデオとは独立して閲覧できます。各モジュールフォルダーには`studio`フォルダーも含まれており、LangGraphアプリケーション構築のためのIDE である[LangGraph Studio](https://github.com/langchain-ai/langgraph-studio)に読み込めるグラフのセットが含まれています。

## セットアップ

開始する前に、`README`の指示に従って環境を作成し、依存関係をインストールしてください。

## Chat models

このコースでは[Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models)を使用します。これらはメッセージのシーケンスを入力として受け取り、チャットメッセージを出力として返すものです。LangChainはChat Modelsをホストしておらず、サードパーティとの統合に依存しています。[こちら](https://python.langchain.com/v0.2/docs/integrations/chat/)がLangChain内のサードパーティチャットモデル統合のリストです！デフォルトでは、コースは人気があり高性能な[ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/)を使用します。前述のとおり、`OPENAI_API_KEY`を確実に設定してください。

`OPENAI_API_KEY`が設定されているかどうかを確認し、設定されていない場合は入力を求められます。

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

[こちら](https://python.langchain.com/v0.2/docs/how_to/#chat-models)がチャットモデルで実行できるすべての操作に関する有用なハウツーですが、以下でいくつかのハイライトを示します。READMEに記載されているように`pip install -r requirements.txt`を実行した場合、`langchain-openai`パッケージがインストールされています。これにより、`ChatOpenAI`モデルオブジェクトをインスタンス化できます。初めてAPIに登録する場合、任意のモデルに適用できる[無料クレジット](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517)を受け取るはずです。さまざまなモデルの価格は[こちら](https://openai.com/api/pricing/)で確認できます。ノートブックはデフォルトで`gpt-4o`を使用します。これは品質、価格、速度の良いバランスを提供するためです（[詳細はこちら](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini)）が、より低価格の`gpt-3.5`シリーズモデルを選択することもできます。

チャットモデルで設定できる[いくつかの標準パラメータ](https://python.langchain.com/v0.2/docs/concepts/#chat-models)があります。最も一般的な2つは：

* `model`: モデルの名前
* `temperature`: サンプリング温度

`Temperature`はモデル出力のランダム性や創造性を制御します。低い温度（0に近い）はより決定論的で焦点を絞った出力を生成します。これは正確性や事実的な応答を必要とするタスクに適しています。高い温度（1に近い）は創造的なタスクや多様な応答の生成に適しています。

In [2]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

LangChainのチャットモデルには多くの[デフォルトメソッド](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface)があります。ほとんどの場合、以下を使用します：

* `stream`: 応答のチャンクをストリームバック
* `invoke`: 入力でチェーンを呼び出し

また、前述のとおり、チャットモデルは[メッセージ](https://python.langchain.com/v0.2/docs/concepts/#messages)を入力として受け取ります。メッセージには役割（メッセージを言っている人を記述）とコンテンツプロパティがあります。これについては後でもっと詳しく説明しますが、ここでは基本を示しましょう。

In [3]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gpt4o_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-d3c4bc85-ef14-49f6-ba7e-91bf455cffee-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

`AIMessage`レスポンスが得られます。また、チャットモデルを文字列で直接呼び出すこともできることに注意してください。文字列が入力として渡されると、`HumanMessage`に変換されてから基盤となるモデルに渡されます。

In [4]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-d6f6b682-e29a-44de-b45e-79fad1e405e5-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [5]:
gpt35_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c75d3f0f-2d71-47be-b14c-42b8dd2b4b08-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

インターフェースはすべてのチャットモデルで一貫しており、モデルは通常各ノートブックの開始時に一度初期化されます。

そのため、他のプロバイダーに強い好みがある場合、下流のコードを変更することなく簡単にモデルを切り替えることができます。

## 検索ツール

READMEには[Tavily](https://tavily.com/)も記載されています。これはLLMとRAG用に最適化された検索エンジンで、効率的で迅速かつ持続的な検索結果を目指しています。前述のとおり、サインアップは簡単で、寛大な無料ティアを提供しています。一部のレッスン（モジュール4）ではデフォルトでTavilyを使用しますが、もちろん自分でコードを修正したい場合は他の検索ツールも使用できます。

In [10]:
_set_env("TAVILY_API_KEY")

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("What is LangGraph?")

In [7]:
search_docs

[{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
  'content': 'LangGraph is a library within the LangChain ecosystem designed to tackle these challenges head-on. LangGraph provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured manner.'},
 {'url': 'https://langchain-ai.github.io/langgraph/',
  'content': 'Overview LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence. LangGraph allows you to define flows that involve cycles, essential for most agentic architectures, differentiating it from DAG-based solutions. As a ...'},
 {'url': 'https://www.youtube.com/watch?v=nmDFSVRnr4Q',
  'content': 'LangGraph is an extension of LangChain enabling Multi-Agent conversation and cyclic chains. This video explains the basics of LangGraph and codesLang